In [57]:
import pandas as pd
import zipfile

zf = zipfile.ZipFile('C:/Users/91971/Desktop/Python Notebooks/NLP Project/archive.zip') # Add path to zip file 
df = pd.read_csv(zf.open('kaggle_RC_2019-05.csv')) # Add name of .csv file tp read

In [58]:
# Import necessary models and libraries for the Pre Processing
import nltk
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer() # For lemmatization of text 
nltk.download('wordnet')

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r"\w{2,}") # For tokenization, matches two or more word characters consecutively

from spacy.lang.en import English
nlp = English() # For providing information about the English words in text


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91971\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [59]:
df = df.dropna(subset=['body']) # Drop NA valued 'body' rows

# Convert all strings in 'body' to lowercase and replace all the characters matching the following regex with no space
df['body'] = df['body'].str.lower() 
df['body'] = df['body'].str.replace(r"""([@&]\w+.*?[:;]?"|â€œ.*?|â€.*?|[()\[\]{}*+\|.,;:<>!#$%^_=~-])""", '')

# Drop columns of no use
df = df.drop('controversiality',axis=1)
df = df.drop('score',axis=1)

In [60]:
# Iterate through all rows of the dataset to filter the 'body' column

for i, row in df.iterrows():

    # Tokenize the 'body' 
    tokenize_body = tokenizer.tokenize(df['body'].iloc[i])

    # Remove stop words and punctuations(if any are left after tokenization) from 'body'
    new_body = []
    final_body = []
    for word in tokenize_body:
        word = lemma.lemmatize(word)
        lexeme = nlp.vocab[word]
        if (lexeme.is_stop == False) and (lexeme.is_punct == False):
            new_body.append(word)
    for word in new_body:
        word = lemma.lemmatize(word)
        final_body.append(word)

    # Join all filtered tokens of 'body' into a new string
    df.at[i,'body'] = ' '.join(final_body)

In [61]:
# Group the filtered text based on 'subreddit' column by joining all text beloning to same entry
df['body'] = df.groupby('subreddit')['body'].transform(lambda x: ' '.join(x))

# Drop the redundant rows 
df.drop_duplicates(subset=['subreddit'], inplace=True)

In [81]:
# Save Pre Processed data to a .csv file 
df.to_csv('pre-processed-data.csv')